In [ ]:
# Credits for the code DeepLearning.AI
!pip install jsonlines
!pip install datasets

In [ ]:
!pip install transformers
!pip install accelerate -U

In [ ]:
# pip install lamini

In [ ]:
import itertools
import jsonlines

from datasets import load_dataset
from pprint import pprint

# from llama import BasicModelRunner
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,Trainer

In [ ]:
import pandas as pd
df=pd.read_excel("/content/Autograde_Dataset.xlsx")
df=df.fillna("Overall Feedback\nNice report!")
df=df.drop("Homework",axis=1)
df.columns

Index(['Task', 'Task_Solution_Code', 'Task_Solution_Report',
       'Task_Solution_Expectations', 'Task_Solution_Result_Points',
       'Task_Solution_Result_Feedback'],
      dtype='object')

In [ ]:
instruction_tuned_dataset = df#load_dataset("squad_v2", split="train")#, streaming=True)

In [ ]:
# load_dataset("squad_v2")

In [ ]:
# m = 1200
# print("Instruction-tuned dataset:")
top_m = df#list(itertools.islice(instruction_tuned_dataset, m))
# for j in top_m:
#   print(top_m[j])

In [ ]:
# top_m

In [ ]:
prompt= """ Given the task{task}\ncode{code}\nreport{report}\ntask_expectations{task_expectations}\n
Provide a feedback:
"""

# prompt_template_without_input = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

# ### Instruction:
# {instruction}

# ### Response:"""

In [ ]:
processed_data = []
for j in range(len(top_m)):
  processed_prompt = prompt.format(task=df["Task"][j],code=df["Task_Solution_Code"][j],report=df["Task_Solution_Report"][j],
                                   task_expectations=df["Task_Solution_Expectations"][j])
  # else:
    # processed_prompt = prompt_template_with_input.format(instruction=j["instruction"], input=j["input"])

  processed_data.append({"input": processed_prompt, "output": df["Task_Solution_Result_Feedback"][j]})


In [ ]:
# processed_data[0]

In [ ]:
with jsonlines.open(f'my_squad.jsonl', 'w') as writer:
    writer.write_all(processed_data)

In [ ]:
# dataset_path_hf = "lamini/alpaca"
# dataset_hf = load_dataset(dataset_path_hf)
# print(dataset_hf)

In [ ]:
# non_instruct_model = BasicModelRunner("meta-llama/Llama-2-7b-hf")
# non_instruct_output = non_instruct_model("Tell me how to train my dog to sit")
# print("Not instruction-tuned output (Llama 2 Base):", non_instruct_output)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m") #("EleutherAI/pythia-70m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")#("EleutherAI/pythia-70m")
model_name="EleutherAI/pythia-70m"

In [ ]:
def inference(text, model, tokenizer, max_input_tokens=514, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [ ]:
finetuning_dataset_path = "squad_v2"
finetuning_dataset = load_dataset(finetuning_dataset_path)
print(finetuning_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})


In [ ]:
test_sample = finetuning_dataset["validation"][0]
print(test_sample["question"])

print(inference(test_sample["question"], model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In what country is Normandy located?


The only way to get to Normandy is to get to the top of the mountain.

The only way to get to Normandy is to get to the top of the mountain.

The only way to get to Normandy is to get to the top of the mountain.

The only way to get to Normandy is to get to the top of the mountain.

The only way to get to Normandy is to


In [ ]:
# Pssst! If you were curious how to upload your own dataset to Huggingface
# Here is how we did it

# !pip install huggingface_hub
# !huggingface-cli login

# import pandas as pd
# import datasets
# from datasets import Dataset

# finetuning_dataset = Dataset.from_pandas(pd.DataFrame(data=finetuning_dataset))
# finetuning_dataset.push_to_hub(dataset_path_hf)

In [ ]:
# def tokenize_function(examples):
#     text = examples["input"][0] + examples["output"][0]["text"][0]


#     tokenizer.pad_token = tokenizer.eos_token
#     tokenized_inputs = tokenizer(
#         text,
#         return_tensors="np",
#         padding=True,
#     )

#     max_length = min(
#         tokenized_inputs["input_ids"].shape[1],
#         2048
#     )
#     tokenizer.truncation_side = "right"
#     tokenized_inputs = tokenizer(
#         text,
#         return_tensors="np",
#         truncation=True,
#         max_length=2048,
#         padding="max_length"
#     )

#     return tokenized_inputs

def tokenize_function(examples):
    # Combine context and question for input
    text_input = examples["input"][0]
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    # Tokenize input (context + question)
    tokenized_inputs = tokenizer(
        text_input,
        return_tensors="np",
        truncation=True,
        max_length=2048,
        padding="max_length"
    )

    # Tokenize output (answer)
    text_output = examples["output"][0]
    tokenized_output = tokenizer(
        text_output,
        return_tensors="np",
        truncation=True,
        max_length=2048,
        padding="max_length"
    )

    # Update tokenized_inputs with tokenized_output for labels
    tokenized_inputs["labels"] = tokenized_output["input_ids"]

    return tokenized_inputs



In [ ]:
# !pip install utilities

In [ ]:
import datasets
import tempfile
import logging
import random
# import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines

# from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
# from llama import BasicModelRunner


logger = logging.getLogger(__name__)
global_config = None

In [ ]:
# s=5
# sample_5 = list(itertools.islice(instruction_tuned_dataset, s))
# for j in sample_5:
#   print(j)
finetuning_dataset_loaded = datasets.load_dataset("json", data_files="/content/my_squad.jsonl", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
finetuning_dataset_loaded["output"][0]

'Overall Feedback\nNice report!'

In [ ]:
# finetuning_dataset_loaded = datasets.load_dataset("json", data_files="squad_v2", split="train")

tokenized_dataset = finetuning_dataset_loaded.map(
    tokenize_function,
    batched=True,
    batch_size=1,
    drop_last_batch=True
)

print(tokenized_dataset)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5
})


In [ ]:
# tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4
    })
    test: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
})


In [ ]:
dataset_name = "my_squad.jsonl"
dataset_path = f"/content/{dataset_name}"
use_hf = False

In [ ]:
model_name="deepset/tinyroberta-squad2"

In [ ]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
# train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)
train_dataset, test_dataset =split_dataset
print(train_dataset)
print(test_dataset)

train
test


In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at deepset/tinyroberta-squad2 and are newly initialized: ['lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

In [ ]:
base_model.to(device)

RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [ ]:
split_dataset[test_dataset][0]['output']
#split_dataset[test_dataset][0]['output']['text'][0]

'Overall Feedback\n-2 (Report is missing discussions on improvements or degradations of accuracy when you tried updating create_better_vectors)'

In [ ]:
test_text = split_dataset[test_dataset][0]['input']
print("Question input (test):", test_text)
print(f"Correct answer : {split_dataset[test_dataset][0]['output']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

Question input (test):  Given the taskCS 590 NLP
HW5, Word2Vec
Due 10/30 11:59 pm
Overall Goal:
In this homework you will work with word2vec embeddings to understand them and set up a simple
classifier. To save time, you will be again working with the dataset from HW3-4.
Useful python packages:
For word2vec, you may find the gensim package useful:
https://radimrehurek.com/gensim/models/word2vec.html
Note that gensim has built in pretrained word2vec embeddings so you should choose 1 or experiment
with multiple in this homework.
(https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models)
Or use glove embeddings (might require different steps to read them in):
https://nlp.stanford.edu/projects/glove/
Gensim also has nice built-in functions for working with word2vec data.
Another option would be to use Polyglot for embeddings:
https://polyglot.readthedocs.io/en/latest/Embeddings.html
Any other options should be cleared with the professor first.
Dataset
The dataset that will be

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 514, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


RuntimeError: ignored

In [ ]:
max_steps = 2
trained_model_name = f"my_squad_{max_steps}_steps"
output_dir = trained_model_name

In [ ]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=2,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=-1,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=60, # Number of update steps between two evaluations
  save_steps=60, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [ ]:
# model_flops = (
#   base_model.floating_point_ops(
#     {
#        "input_ids": torch.zeros(
#            (1, training_config["model"]["max_length"])
#       )
#     }
#   )
#   * training_args.gradient_accumulation_steps
# )

# print(base_model)
# print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
# print("Flops", model_flops / 1e9, "GFLOPs")

In [ ]:
train_dataset=split_dataset[train_dataset]
test_dataset=split_dataset[test_dataset]

In [ ]:
# from transformers import Trainer
# from utilities import *
# !pip install transformers



In [ ]:
import torch
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Trainer(
    model=base_model,
    # model_flops=model_flops,
    # total_steps=100,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
train_dataset

In [ ]:
training_output = trainer.train()

In [ ]:
training_output

In [ ]:
save_dir = f'{output_dir}/finetuned_2'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

In [ ]:
finetuned_2 = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)


In [ ]:
finetuned_2.to(device)


In [ ]:
test_question = test_dataset[1]['input']
print("Question input (test):", test_question)

print("Finetuned_2 model's answer: ")
print(inference(test_question, finetuned_2, tokenizer))

In [ ]:
test_answer = test_dataset[1]['output']["text"][0]
print("Target answer output (test):", test_answer)